In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import sys
import scipy.stats
from matplotlib.ticker import FormatStrFormatter
from ml_datasets.esl import ProstateCancer
from esl.utils import scale

In [2]:
prostate_cancer = ProstateCancer(verbose=0)

In [3]:
df = prostate_cancer.load()
# extract input and output dataframe
train_test = prostate_cancer.train_test
df_x = df[prostate_cancer.meta[:-1]]
df_y = df[prostate_cancer.meta[-1]]

x_train = scale(df_x[train_test=='T'].copy()).values
y_train =  df_y[train_test=='T'].copy().values

In [4]:
def kfold(x, y, nfold=10):
    num_data = len(y)
    index = np.arange(num_data)
    np.random.seed(2)
    np.random.shuffle(index)
    
    num_data = len(y)
    avg = len(index) / float(nfold)
    last = 0.0
    
    x_dict = dict()
    y_dict = dict()
    
    i = 0
    while last < num_data:
        index_val = index[int(last):int(last + avg)]
#         print(last, avg, int(last + avg))
        index_train = np.array([j for j in range(num_data) if j not in index_val])
        
        y_dict[9 - i] = {'train': y[index_train], 'val': y[index_val]}
        x_dict[9 - i] = {'train': x[index_train, :], 'val': x[index_val, :]}
        
        last += avg
        i += 1
    
    return x_dict, y_dict

In [5]:
# x_dict, y_dict = kfold(x_train, y_train, nfold=10)

In [ ]:
# class Lasso:
    
#     def fit(self, x , y):
        

In [30]:
r = y_train - np.mean(y_train)
beta_hat = np.zeros(x_train.shape[1])
A = np.zeros_like(x_train)
delta = np.zeros_like(beta_hat)
f = np.zeros_like(y_train)
alpha = 0.95

In [36]:
for k in np.arange(1, x_train.shape[1] + 1):
    corr = x_train.T @ r

    corr_best_idx = np.argmax(corr)
    A[:, corr_best_idx] = x_train[:, corr_best_idx]
    
    delta[corr_best_idx] = A[:, corr_best_idx].T @ r
    
    beta_hat[corr_best_idx]  = beta_hat[corr_best_idx]  + alpha * delta[corr_best_idx] 

    f = A[:, corr_best_idx] * beta_hat[corr_best_idx]
    
    r = r - f
    
    u = A[:, corr_best_idx] * delta[corr_best_idx]
    f = f + alpha * u

In [34]:
print(beta_hat)

[ 1.11881477e+02 -1.16249216e+17  0.00000000e+00  3.74581278e+05
  2.26675088e+05  0.00000000e+00  5.87804135e+16  0.00000000e+00]
